In [197]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

# Base

In [307]:
df = pd.read_parquet("data/processed/interpolated_ahei.parquet")
#df.head()

# Prices Database

In [245]:
prices = pd.read_excel("data/raw/prices_for_nutrition.xlsx")
#prices.head()

In [200]:
#prices.tail(10)

In [247]:
prices_calc = prices[:-5]
prices_calc = (
    prices_calc[["Country Code", "Time"] + prices.columns.tolist()[6:]]
    .rename(columns={"Time": "Year"})
    .replace("..", np.nan)
    .interpolate()
    .convert_dtypes()
)
prices_calc

,Country Code,Year,Cost of an energy sufficient diet [CoCA],Cost of a nutrient adequate diet [CoNA],Cost of a healthy diet [CoHD],Cost of a healthy diet relative to the cost of sufficient energy from starchy staples [CoHD_CoCA],Cost of fruits [CoHD_f],Cost of vegetables [CoHD_v],Cost of starchy staples [CoHD_ss],Cost of animal-source foods [CoHD_asf],...,Affordability of a healthy diet: ratio of cost to the $0.99 food poverty line [CoHD_pov],Affordability of an energy sufficient diet: ratio of cost to food expenditures [CoCA_fexp],Affordability of a nutrient adequate diet: ratio of cost to food expenditures [CoNA_fexp],Affordability of a healthy diet: ratio of cost to food expenditures [CoHD_fexp],Percent of the population who cannot afford sufficient calories at 52% of income [CoCA_headcount],Percent of the population who cannot afford nutrient adequacy at 52% of income [CoNA_headcount],Percent of the population who cannot afford a healthy diet at 52% of income [CoHD_headcount],Millions of people who cannot afford sufficient calories at 52% of income [CoCA_unafford_n],Millions of people who cannot afford nutrient adequacy at 52% of income [CoNA_unafford_n],Millions of people who cannot afford a healthy diet at 52% of income [CoHD_unafford_n]
0,ALB,2017,0.725,2.471,3.952,5.45,0.911,0.707,0.599,1.204,...,3.992,0.078,0.266,0.425,0.0,13.0,37.8,0.0,0.4,1.1
1,ALB,2018,0.737,2.434,4.051,5.305,0.928,0.705,0.57325,1.144,...,4.092,0.0895,0.293,0.47,0.05,11.55,27.9,0.025,1.05,0.8
2,ALB,2019,0.749,2.397,4.117,5.16,0.945,0.703,0.5475,1.084,...,4.159,0.101,0.32,0.515,0.1,10.1,19.8,0.05,1.7,0.6
3,ALB,2020,0.761,2.36,4.197,5.015,0.962,0.701,0.52175,1.024,...,4.239,0.1125,0.347,0.56,0.15,8.65,20.1,0.075,2.35,0.6
4,DZA,2017,0.773,2.323,3.763,4.87,0.979,0.699,0.496,0.964,...,3.801,0.124,0.374,0.605,0.2,7.2,35.2,0.1,3.0,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,ZMB,2020,0.901,2.31775,3.3,3.9475,0.47775,0.88375,0.616,0.939,...,3.333,0.446,1.11325,1.602,27.8,73.55,88.0,4.325,11.025,16.2
740,ZWE,2017,0.775,2.297,3.456,4.46,0.513,0.968,0.55,0.95,...,3.491,0.343,1.016,1.529,15.8,70.8,84.1,2.2,10.1,12.0
741,ZWE,2018,0.775,2.297,3.456,4.46,0.513,0.968,0.55,0.95,...,3.491,0.343,1.016,1.529,15.8,70.8,84.1,2.2,10.1,12.0
742,ZWE,2019,0.775,2.297,3.456,4.46,0.513,0.968,0.55,0.95,...,3.491,0.343,1.016,1.529,15.8,70.8,84.1,2.2,10.1,12.0


In [248]:
prc = df.merge(prices_calc, on=["Country Code", "Year"])

In [249]:
x = prc.drop(['AHEI', 'Year', 'Country Code'], axis =1)
y = prc['AHEI']

In [250]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [251]:
from sklearn.feature_selection import RFE

In [252]:
rfe = RFE(model, 7)

C:\Users\Nickolas Machado\miniconda3\envs\onu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=7 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [253]:
fit = rfe.fit(x, y)

C:\Users\Nickolas Machado\miniconda3\envs\onu\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [254]:
cols1 = fit.get_support(indices=True)

In [256]:
feat_price = x.iloc[:,cols1].columns.to_list()
feat_price

['Cost of a healthy diet [CoHD]',
 'Cost share for fruits in a least-cost healthy diet [CoHD_f_prop]',
 'Cost share for vegetables in a least-cost healthy diet [CoHD_v_prop]',
 'Cost share for starchy staples in a least-cost healthy diet [CoHD_ss_prop]',
 'Cost share for animal-sourced foods in a least-cost healthy diet [CoHD_asf_prop]',
 'Cost share for legumes, nuts and seeds in a least-cost healthy diet [CoHD_lns_prop]',
 'Cost share for oils and fats in a least-cost healthy diet [CoHD_of_prop]']

# CPI

In [257]:
cpi = pd.read_csv("data/raw/cpi.csv")

In [258]:
cpi_calc = (
    cpi.query("(FREQUENCY == 'A') and (MEASURE == 'AGRWTH')")
    [["LOCATION", "SUBJECT", "TIME", "Value"]]
    .rename(columns={"LOCATION": "Country Code", "Value": "CPI", "TIME": "Year"})
    .pivot(columns="SUBJECT", values="CPI", index=["Country Code", "Year"])
    .reset_index()
    .convert_dtypes()
    .sort_values(by=["Country Code", "Year"])
)
cpi_calc["Year"] = cpi_calc["Year"].astype("int")
cpi_calc.head()

SUBJECT,Country Code,Year,ENRG,FOOD,TOT,TOT_FOODENRG
0,ARG,2018,<NA>,32.73381,34.27723,<NA>
1,ARG,2019,<NA>,58.36288,53.54831,<NA>
2,ARG,2020,<NA>,46.96305,42.01509,<NA>
3,ARG,2021,<NA>,49.84582,48.40938,<NA>
4,AUS,1950,<NA>,<NA>,8.695652,<NA>


In [259]:
# Food and tot may be promissing
cpp = df.merge(cpi_calc, on=["Country Code", "Year"])#.corr()["AHEI"]
#cpp[cpp['Country Code'] == "BRA"].corr()["AHEI"]

In [260]:
cpp = cpp.dropna()

In [261]:
cpp.head(2)

,Country Code,Year,AHEI,ENRG,FOOD,TOT,TOT_FOODENRG
23,AUS,1972,45.936247,4.91007,3.907289,6.024096,6.458926
24,AUS,1973,45.936247,3.762801,15.35854,9.090909,8.337001


In [262]:
x = cpp.drop(['AHEI', 'Year', 'Country Code'], axis =1)
y = cpp['AHEI']

In [263]:
rfe = RFE(model, 2)

C:\Users\Nickolas Machado\miniconda3\envs\onu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [264]:
fit = rfe.fit(x, y)

C:\Users\Nickolas Machado\miniconda3\envs\onu\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [265]:
cols2 = fit.get_support(indices=True)

In [267]:
feat_cpi = x.iloc[:,cols2].columns.to_list()
feat_cpi

['TOT', 'TOT_FOODENRG']

# WDI

In [270]:
def long_to_wide(df: pd.DataFrame):
    cols = [col for col in df.columns if "20" in col]
    wide = df.rename(columns={"Country Code": "Country"})
    wide = wide.melt(
        id_vars=["Country", "Indicator Name"], value_vars=cols, var_name="Year", value_name="Pct"
    )
    wide = wide.set_index(["Country", "Year"]).pivot(columns="Indicator Name")
    wide.columns = [col[1] for col in wide.columns]
    return wide.reset_index()

In [321]:
wdi = long_to_wide(pd.read_csv("data/raw/world_development_index.csv"))
wdi

,Country,Year,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),...,Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who believe a husband is justified in beating his wife when she argues with him (%),Women who believe a husband is justified in beating his wife when she burns the food (%),Women who believe a husband is justified in beating his wife when she goes out without telling him (%),Women who believe a husband is justified in beating his wife when she neglects the children (%),Women who believe a husband is justified in beating his wife when she refuses sex with him (%),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
0,ABW,2000,NaN,NaN,NaN,NaN,91.660400,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,2001,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABW,2002,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,2003,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABW,2004,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,ZWE,2017,NaN,29.8,6.0,79.3,44.178635,24.531088,85.478287,55.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.9,13000.0
5848,ZWE,2018,NaN,29.9,6.4,79.1,45.572647,26.617121,85.468765,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.1,10000.0
5849,ZWE,2019,48.0,30.1,6.7,79.1,46.781475,28.404877,85.457336,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.418352,33.658057,60.3,8600.0
5850,ZWE,2020,NaN,30.4,6.8,78.6,52.747669,37.060249,85.715477,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.5,7700.0


In [323]:
wdi_calc = (
    wdi.rename(columns={"Country": "Country Code"})
    .convert_dtypes()
)
wdi_calc["Year"] = wdi_calc["Year"].astype("int")

In [324]:
wdi_rfe = df.merge(wdi_calc, on=["Country Code", "Year"])

In [275]:
x = wdi_rfe.drop(['AHEI', 'Year', 'Country Code'], axis =1)
y = wdi_rfe['AHEI']

In [276]:
rfe = RFE(model, 5)

C:\Users\Nickolas Machado\miniconda3\envs\onu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [277]:
fit = rfe.fit(x, y)

In [278]:
cols3 = fit.get_support(indices=True)

In [279]:
feat_wdi = x.iloc[:,cols3].columns.to_list()
feat_wdi

['Human capital index (HCI) (scale 0-1)',
 'Human capital index (HCI), lower bound (scale 0-1)',
 'Human capital index (HCI), male (scale 0-1)',
 'Human capital index (HCI), male, lower bound (scale 0-1)',
 'Human capital index (HCI), upper bound (scale 0-1)']

In [280]:
# the other features are describes the same thing
feat_wdi = feat_wdi[:1]
feat_wdi

['Human capital index (HCI) (scale 0-1)']

In [325]:
data_final = df.copy()
for feat_df, feat_cols in zip([wdi_rfe, cpi_calc, prices_calc], [feat_wdi, feat_cpi, feat_price]):
    data_final = data_final.merge(
        feat_df[["Country Code", "Year"] + feat_cols], on=["Country Code", "Year"], how="outer"
    )
data_final.columns

Index(['Country Code', 'Year', 'AHEI', 'Human capital index (HCI) (scale 0-1)',
       'TOT', 'TOT_FOODENRG', 'Cost of a healthy diet [CoHD]',
       'Cost share for fruits in a least-cost healthy diet [CoHD_f_prop]',
       'Cost share for vegetables in a least-cost healthy diet [CoHD_v_prop]',
       'Cost share for starchy staples in a least-cost healthy diet [CoHD_ss_prop]',
       'Cost share for animal-sourced foods in a least-cost healthy diet [CoHD_asf_prop]',
       'Cost share for legumes, nuts and seeds in a least-cost healthy diet [CoHD_lns_prop]',
       'Cost share for oils and fats in a least-cost healthy diet [CoHD_of_prop]'],
      dtype='object')

In [326]:
data_final.columns = [
    "Country", "Year", "AHEI", "HCI", "CPI", "CPIFoodEnergy",
    "HealthyDietCost", "FruitsShare", "VegetablesShare", "StarchyShare",
    "AnimalSourceShare", "LegumesShare", "OilsShare",
]
data_final

,Country,Year,AHEI,HCI,CPI,CPIFoodEnergy,HealthyDietCost,FruitsShare,VegetablesShare,StarchyShare,AnimalSourceShare,LegumesShare,OilsShare
0,AFG,1990,51.369047,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,AFG,1991,51.246402,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,AFG,1992,51.123757,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,AFG,1993,51.001112,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,AFG,1994,50.878467,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7166,WLD,2020,NaN,<NA>,<NA>,<NA>,3.537,0.131875,0.2005,0.25525,0.299375,0.072625,0.041375
7167,ZMB,2019,NaN,<NA>,<NA>,<NA>,3.245,0.1345,0.2425,0.2115,0.2845,0.085,0.042
7168,ZMB,2020,NaN,<NA>,<NA>,<NA>,3.3,0.14125,0.26125,0.18525,0.27975,0.09,0.042
7169,ZWE,2019,NaN,<NA>,<NA>,<NA>,3.456,0.148,0.28,0.159,0.275,0.095,0.042


In [327]:
data_final.to_parquet("data/processed/features.parquet")